# K Nearest Neighbor
| Colin Yee

In [1]:
from cifar10_import import *
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid

#### Determining Dimensionality Loss

The biggest hyperparameters that I have to figure out is k and how to do dimensional reductionality the best way

Cross Validation

In [2]:
# Import the CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = importCifar10()
assert x_train.shape == (50000, 32, 32, 3) # For slot in the 32 x 32 grid, it's an list of 3 numbers for RGB
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [14]:
x_train_2d = x_train.flatten().reshape(50000, 3072)
x_test_2d = x_test.flatten().reshape(10000,3072)

(50000, 3072)

In [10]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train_2d,y_train)


c:\Users\colin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


<bound method ClassifierMixin.score of KNeighborsClassifier()>

In [15]:
y_pred = knn.predict(x_test_2d)
1 - accuracy_score(y_test,y_pred)

0.6602